## **Figure 3**: Effect of Spatial Resolution on Noise in SWMMIN Simulations

#### Importing required libraries

In [1]:
import pyswmm                   # Library for working with SWMM (Storm Water Management Model)
import pandas as pd             # Data manipulation and analysis library
import numpy as np              
import pathlib                  # Object-oriented filesystem paths
import matplotlib as mpl        
import matplotlib.pyplot as plt  

# Set font properties for matplotlib
font = {'family' : 'Serif',
        'weight' : 'bold', 
        'size'   : 10}
mpl.rc('font', **font)  # Apply font settings to matplotlib
mpl.rc('font', **font)  # Apply font settings to matplotlib (duplicate line, can be removed)
mpl.rcParams['figure.dpi'] = 600  # Set the resolution of the figure
mpl.rcParams["mathtext.fontset"] = "dejavuserif"  # Set the font for math text

In [2]:
# Directory for network inp files
dir = pathlib.Path('../Networks/Linear Network')
# FIle names for network inp files
partial_name = 'Linear_Network_SWMMIN_'
filenames = ['1000m.inp','100m.inp','10m.inp','ASD.inp']

In [3]:
stp=0       #steps counter
every=10000  #Interval of printing current time

# Running the simulation for each network
for filename in filenames:
    file_path = dir.joinpath(partial_name+ filename)
    print(file_path)
    sim = pyswmm.Simulation(inputfile=str(file_path), outputfile=str(file_path).replace('.inp','.out'))
    
    with sim as sim:
        for step in sim:
            if stp%every==0:
                print('Current Time is >> ',sim.current_time)
            stp+=1
            pass

..\Networks\Linear Network\Linear_Network_2_SWMMIN_1000m.inp
Current Time is >>  2022-05-18 00:00:00
Current Time is >>  2022-05-18 02:46:40
Current Time is >>  2022-05-18 05:33:20
Current Time is >>  2022-05-18 08:20:00
Current Time is >>  2022-05-18 11:06:40
Current Time is >>  2022-05-18 13:53:20
Current Time is >>  2022-05-18 16:40:00
Current Time is >>  2022-05-18 19:26:40
Current Time is >>  2022-05-18 22:13:20
..\Networks\Linear Network\Linear_Network_2_SWMMIN_100m.inp
Current Time is >>  2022-05-18 01:01:00
Current Time is >>  2022-05-18 03:47:40
Current Time is >>  2022-05-18 06:34:20
Current Time is >>  2022-05-18 09:21:00
Current Time is >>  2022-05-18 12:07:40
Current Time is >>  2022-05-18 14:54:20
Current Time is >>  2022-05-18 17:41:00
Current Time is >>  2022-05-18 20:27:40
Current Time is >>  2022-05-18 23:14:20
..\Networks\Linear Network\Linear_Network_2_SWMMIN_10m.inp
Current Time is >>  2022-05-18 01:59:56
Current Time is >>  2022-05-18 04:46:36
Current Time is >>  

In [3]:
# Initialize empty DataFrames to store pressures and flowrates
pressures = pd.DataFrame()
flowrates = pd.DataFrame()

# Iterate over each filename in the list of filenames
for filename in filenames:
    last = -1  # Initialize the variable to track the last link or node
    file_path = dir.joinpath(partial_name + filename).with_suffix('.out')  # Construct the file path
    print(file_path)  # Print the file path for debugging purposes
    
    # Open the SWMM output file
    with pyswmm.Output(str(file_path)) as output:
        # Iterate over each link in the output
        for link in output.links:
            link_name = link.split('-')  # Split the link name by '-'
            if link_name[0] == 'P1':  # Check if the link starts with 'P1'
                if int(link_name[1]) > last:  # Update the last link if the current link number is greater
                    last = int(link_name[1])
        link_target = 'P1-' + str(last)  # Construct the target link name
        # Store the flow rate series in the flowrates DataFrame
        flowrates[filename.replace('.inp', '')] = output.link_series(link_target, 'Flow_rate').values()
        
        last = -1  # Reset the variable to track the last node
        # Iterate over each node in the output
        for node in output.nodes:
            node_name = node.split('-')  # Split the node name by '-'
            if node_name[0] == '1':  # Check if the node starts with '1'
                if int(node_name[1]) > last:  # Update the last node if the current node number is greater
                    last = int(node_name[1])
        if last == -1:
            node_target = 'DN1'  # Set the target node to 'DN1' if no nodes were found
        else:
            node_target = '1-' + str(last) + '-DN1'  # Construct the target node name
        # Store the depth series in the pressures DataFrame
        pressures[filename.replace('.inp', '')] = output.node_series(node_target, 'DEPTH').values()
        # Extract the time series from the depth series
        time = list(output.node_series(node_target, 'DEPTH').keys())

..\Networks\Linear Network\Linear_Network_2_SWMMIN_1000m.out
..\Networks\Linear Network\Linear_Network_2_SWMMIN_100m.out
..\Networks\Linear Network\Linear_Network_2_SWMMIN_10m.out
..\Networks\Linear Network\Linear_Network_2_SWMMIN_ASD.out


In [6]:
pressures.max()

1000m    28.065231
100m     12.294351
10m       6.165339
ASD       6.133114
dtype: float64

In [4]:
time = pd.Series(time)
time_minutes = (time - time[0]).dt.total_seconds() / 60  # Convert to float hours
list_of_delxs=["Original","500m","250m","100m","50m","10m"]
font = {'family' : 'Serif',
        'weight' : 'bold', 
        'size'   : 12}
mpl.rc('font', **font)

fig,ax=plt.subplots(2,4,sharex=True)
fig.set_figheight(11)
fig.set_figwidth(20)

ax[0,0].plot(time_minutes,flowrates['1000m'],label="1,000 m",c="#e66101")
ax[1,0].plot(time_minutes,pressures[('1000m')],c="#e66101")
ax[0,1].plot(time_minutes,flowrates["100m"],label="100 m",c="#fdb863")
ax[1,1].plot(time_minutes,pressures["100m"],c="#fdb863")
ax[0,2].plot(time_minutes,flowrates["10m"],label="10 m",c="#b2abd2")
ax[1,2].plot(time_minutes,pressures["10m"],c="#b2abd2")
ax[0,3].plot(time_minutes,flowrates["ASD"],label="Pachaly et al. (2020)'s ASD",c="#5e3c99")
ax[1,3].plot(time_minutes,pressures["ASD"],c="#5e3c99")
xlim1=(0,50)
for i in [0,1,2,3]:
    ax[1,i].set_ylim(0,15)
    ax[1,i].set_yticks(np.arange(0,15.1,3))
    ax[0,i].set_ylim(0,300)

ax[0, 0].set_title("Original (1,000 m)")
ax[0, 1].set_title("100 m")
ax[0, 2].set_title("10 m")
ax[0, 3].set_title("Pachaly et al. (2020)'s ASD")

ax[0,0].set_ylabel("Flow (LPS)")
ax[1,0].set_ylabel("Pressure (m)")

ax[1,0].annotate(str(round(pressures['1000m'].max(),1))+' m', xy=(10,12),xycoords = 'data', xytext = (15,12), arrowprops = dict(arrowstyle="->"))
# fig.legend(title = "$\Delta x$",loc="lower center",frameon=True,ncol=2, fontsize = 12, bbox_to_anchor = (0.5,-0.02,0,0))
plt.setp(ax, xlim=xlim1,xlabel="Time (min)")
plt.savefig("../../../Figure Files/Pressure-Spikes",dpi=600)
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>